# THE BATTLE OF NEIGHBORHOODS FROM COURSERA CAPSTONE

By Gastón Costa

# Introduction

The problem:

Hello my name is Gastón I am from Argentina but I have been living in Barcelona for 10 years. I have a degree in food science and technology.
Not long ago in my family a member has developed a food allergy, specifically gluten.

Within the day to day at home it is easier to change the way you cook and adapt.

The problem we encounter is when it comes to being away from home. We are lovers of breakfast and snacks, so lately it is difficult to find places where you can have a quiet breakfast since gluten is present in many foods. We also found the problem the same problem when traveling to other cities and countries.

Business Model:

Through this work, I want to help people who suffer from a food allergy to gluten or who are celiac to be able to draw a map of places where they can enjoy a meal or breakfast without problems.
I will apply it to the city I live in and in Toronto as an example of another city we will visit.

# Data

Description of the data and how it will be used to solve the problem
The following data will be used:

- List of Boroughs and neighborhoods of Barcelona with their geodata;
- List of Boroughs and neighborhoods of Toronto with their geodata;
- List of place whit gluten free description of Barcelona with their geodata;
- List of place whit gluten free description of Toronto with their geodata.

# Data Sources

- Boroughs and neighborhoods of Barcelona from Wikipedia (https://es.wikipedia.org/wiki/Distritos_de_Barcelona);
- Boroughs and neighborhoods of Totonto from Wikipedia (https://es.wikipedia.org/wiki/%C3%81rea_metropolitana_de_Toronto);
- Geocode information from Geopy;
- Gluten Free in Barcelona and Toronto from Foursquare

# Methodology


- The neighborhood where i live in Barcelona will be individuated
- The number of gluten free shops will be extrapolated.
- Different neighborhoods in Toronto we'll be anallyzed and we'll select just the ones with at least the same amount of gluten free shop present in my actual neighborhood
- A map with places to eat will make things easier to eat.

# Importing libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install folium
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans